### Tabnet

In [1]:
# 패키지 불러오기
import pandas as pd
import numpy as np
import time
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import torch

import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

In [2]:
# 데이터 불러오기
x_train = pd.read_csv('../data/fill_data/x_train.csv')
x_valid = pd.read_csv('../data/fill_data/x_valid.csv')
y_train = pd.read_csv('../data/fill_data/y_train.csv')
y_valid = pd.read_csv('../data/fill_data/y_valid.csv')

In [3]:
x_train = x_train.values
x_valid = x_valid.values
y_train = y_train.values
y_valid = y_valid.values

In [4]:
y_train = np.array(y_train).ravel()
y_valid = np.array(y_valid).ravel()

modeling

In [5]:
start_time = time.time()
clf = TabNetClassifier(
    n_d=56, 
    n_steps=3, gamma=1.4,
    lambda_sparse=1.1694994589239397e-05, 
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
    mask_type='sparsemax',
    n_shared=2,
    scheduler_params=dict(mode="min",
        patience = 3, # changing sheduler patience to be lower than early stopping patience
        min_lr=1e-5,factor=0.5,),
    scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
    verbose=0)
clf.fit(x_train, y_train,
    eval_set=[(x_train, y_train),(x_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['accuracy'],
    drop_last = False,
    patience = 29, max_epochs = 80
)
end_time = time.time()
print('걸린 시간 :', end_time - start_time)

## 기본값: 0.81477


Early stopping occurred at epoch 42 with best_epoch = 13 and best_valid_accuracy = 0.82365


/Users/yschoi/bitamin/ChurnProject/venv/lib/python3.7/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


걸린 시간 : 775.2704820632935


In [7]:
# prediction and evaluation
# train data
train_pred_y = clf.predict(x_train)
print('Accuracy: ', (accuracy_score(y_train, train_pred_y)))
print('f1_score: ', f1_score(y_train, train_pred_y))

# valid data
valid_pred_y = clf.predict(x_valid)
print('Accuracy: ', (accuracy_score(y_valid, valid_pred_y)))
print('f1_score: ', f1_score(y_valid, valid_pred_y))

Accuracy:  0.8240567988140021
f1_score:  0.9012676853423883
Accuracy:  0.8236545682102628
f1_score:  0.9008514530997115


하이퍼파라미터 튜닝

In [8]:
def Objective(trial : Trial, x_train, y_train, x_valid, y_valid):
    mask_type = trial.suggest_categorical("mask_type", ["entmax", "sparsemax"])
    n_da = trial.suggest_int("n_da", 56, 64, step=4)
    n_steps = trial.suggest_int("n_steps", 1, 3, step=1)
    gamma = trial.suggest_float("gamma", 1., 1.4, step=0.2)
    n_shared = trial.suggest_int("n_shared", 1, 3)
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True)
    tabnet_params = dict(n_d=n_da, n_a=n_da, n_steps=n_steps, gamma=gamma,
        lambda_sparse=lambda_sparse, optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
        mask_type=mask_type, n_shared=n_shared,
        scheduler_params=dict(mode="min",
        patience=trial.suggest_int("patienceScheduler",low=3,high=10), # changing sheduler patience to be lower than early stopping patience
        min_lr=1e-5,factor=0.5,),
        scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
        verbose=0,
        ) #early stopping
    
    clf = TabNetClassifier(**tabnet_params)
    clf.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)],
        patience=trial.suggest_int("patience",low=15,high=30), max_epochs=trial.suggest_int('epochs', 1, 100),
        eval_metric=['accuracy'])

    score = f1_score(clf.predict(x_valid), y_valid)
    
    return score 


In [ ]:
study = optuna.create_study(direction="maximize", sampler = TPESampler(), study_name='TabNet optimization')
study.optimize(lambda trial : Objective(trial, x_train, y_train, x_valid, y_valid), n_trials = 10)
print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

[I 2023-01-31 22:39:37,221] A new study created in memory with name: TabNet optimization
